In [1]:
# Imports

import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import psycopg2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import pearsonr
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV


pd.set_option('display.max_rows', None) #magic command
pd.set_option('display.max_columns', None)



In [2]:
conn = psycopg2.connect("dbname=mimic user=mimic password=Mimic@4@plhi")
cur = conn.cursor()

In [3]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "Machine_learning_ch-4"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

df_labevents_1 = create_pandas_table("""select itemid, label, loinc_code from mimic_hosp.d_labitems """)
print (df_labevents_1.shape)
df_labevents_1.head(1625)
# df_labevents_1['loinc_code'].isnull().sum

(1625, 3)


,itemid,label,loinc_code
0,51898,,None
1,51532,11-Deoxycorticosterone,None
2,51952,17-Hydroxycorticosteroids,None
3,51953,"17-Ketosteroids, Urine",None
4,52063,24 Hr,None
5,51066,24 hr Calcium,None
6,51067,24 hr Creatinine,None
7,51068,24 hr Protein,None
8,50853,25-OH Vitamin D,None
9,51533,3t,None


In [5]:
# dropping the Null values
df_labevents_1.dropna(inplace = True)
df_labevents_1

,itemid,label,loinc_code
12,51535,5' Nucleotidase,1690-7
17,52064,Absolute Basophil Count,704-7
20,51130,Absolute CD3 Count,8124-0
21,51131,Absolute CD4 Count,8128-1
23,51132,Absolute CD8 Count,8138-0
24,52068,Absolute Eosinophil Count,711-2
27,51133,Absolute Lymphocyte Count,731-0
29,52069,Absolute Monocyte Count,742-7
31,52070,Absolute Neutrophil Count,751-8
34,51134,Acanthocytes,7789-1


In [6]:
# df_labevents = create_pandas_table("""select subject_id, hadm_id, itemid, flag, value from mimic_hosp.labevents""")
# print (df_labevents.shape)
# df_labevents.head(10) 

In [7]:
# df_labevents.to_csv('df_labevents.csv', index = False)

In [8]:
df_labevents = pd.read_csv('df_labevents.csv')

/opt/tljh/user/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
itemid_label = df_labevents_1.set_index('itemid').to_dict()['label']
itemid_label

{51535: "5' Nucleotidase",
 52064: 'Absolute Basophil Count',
 51130: 'Absolute CD3 Count',
 51131: 'Absolute CD4 Count',
 51132: 'Absolute CD8 Count',
 52068: 'Absolute Eosinophil Count',
 51133: 'Absolute Lymphocyte Count',
 52069: 'Absolute Monocyte Count',
 52070: 'Absolute Neutrophil Count',
 51134: 'Acanthocytes',
 50856: 'Acetaminophen',
 50857: 'Acetone',
 50858: 'Acid Phosphatase',
 51539: 'Acid Phosphatase, Prostatic Fraction',
 51540: 'Adrenocorticotrophic Hormone',
 50861: 'Alanine Aminotransferase (ALT)',
 51542: '(Albumin)',
 50862: 'Albumin',
 51070: 'Albumin/Creatinine, Urine',
 51544: 'Aldosterone',
 50863: 'Alkaline Phosphatase',
 51545: 'Alpha-1',
 51547: 'Alpha-2',
 50864: 'Alpha-Fetoprotein',
 51462: 'Amorphous Crystals',
 50867: 'Amylase',
 50868: 'Anion Gap',
 51137: 'Anisocytosis',
 51556: 'Anti-Microsomal Antibodies',
 51557: 'Anti-Microsomal Antibody',
 50873: 'Anti-Nuclear Antibody',
 50874: 'Anti-Nuclear Antibody, Titer',
 50877: 'Anti-Thyroglobulin Antibodi

In [10]:
df_labevents['label'] = df_labevents['itemid'].map(itemid_label)

In [11]:
df_labevents.head(10)

,subject_id,hadm_id,itemid,flag,value,label
0,16047719,NaN,51492,NaN,NaN,Protein
1,16117323,24152682.0,51492,NaN,NaN,Protein
2,16117323,23463879.0,51492,NaN,NaN,Protein
3,16117323,24152682.0,51492,NaN,NaN,Protein
4,16117323,24152682.0,51492,NaN,NaN,Protein
5,16045617,NaN,51492,NaN,NaN,Protein
6,12403460,NaN,51492,NaN,NaN,Protein
7,12403460,NaN,51492,NaN,NaN,Protein
8,12403460,NaN,51492,NaN,NaN,Protein
9,12403460,NaN,51492,NaN,NaN,Protein


In [12]:
df_labevents.shape

(122289828, 6)

In [13]:
df_labevents.dropna(inplace = True)

In [14]:
df_labevents.shape

(19736755, 6)

In [15]:
# # df_labevents_ = df_labevents.loc[df_labevents['label'].isin(['Absolute Lymphocyte Count', 'Absolute Neutrophil Count', 'WBC',  'RBC', 'RDW', 'Basophils', 'Eosinophils','Neutrophils', 'Monocytes',
#                                                                   'Hematocrit', 'Hemoglobin', 'MCV', 'Platelet Count', 'Alanine Aminotransferase (ALT)', 'Asparate Aminotransferase (AST)', 'Alkaline Phosphatase','PT', 'Albumin', 'Globulin',
#                                                                  'Bilirubin, Total', 'Potassium', 'Sodium', 'Creatine Kinase (CK)', 'Cholesterol, HDL', 'Cholesterol, LDL, Calculated', 'Cholesterol, Total', 'C-Reactive Protein', 
#                                                                    'Creatinine', 'Urea Nitrogen', 'Uric Acid', 'Lactate Dehydrogenase (LD)',
#                                                     ])]

In [16]:
df_labevents.dtypes


subject_id      int64
hadm_id       float64
itemid          int64
flag           object
value          object
label          object
dtype: object

In [17]:
df_labevents['flag'] = df_labevents['flag'].fillna('normal')

In [18]:
df_labevents.isnull().sum()


subject_id    0
hadm_id       0
itemid        0
flag          0
value         0
label         0
dtype: int64

In [19]:
# x = pd.read_csv('df_admissions_data.csv')

In [20]:
# x.head(10)

In [21]:
# x.drop(['blood', 'circulatory','congenital', 'digestive', 'endocrine', 'genitourinary', 'infectious', 'injury', 'mental', 'misc','muscular','neoplasms', 'nervous', 'pregnancy', 'prenatal', 'respiratory', 'skin'], axis=1)
# x.head(10)

In [22]:
df_labevents['value'].value_counts()

1.2                                                                            198987
1.3                                                                            196023
21                                                                             182639
1.4                                                                            160122
1.5                                                                            151376
8.3                                                                            126054
8.2                                                                            118545
0.00                                                                           113685
8.1                                                                            110393
8.0                                                                            101773
1.6                                                                            101472
2.6                                                   

In [23]:
# convert datatype of values in column 'value' :- https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
df_labevents["value"] = pd.to_numeric(df_labevents["value"], downcast = 'integer',  errors='coerce') # coerce invalid values to NaN as follows using the errors keyword argument

In [24]:
#Reshaping by pivoting DataFrame objects :- https://pandas-docs.github.io/pandas-docs-travis/user_guide/reshaping.html
df_labevents = df_labevents.pivot_table(index="subject_id", columns="label", values="value")

In [25]:
df_labevents.shape

(196833, 162)

In [26]:
df_labevents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196833 entries, 10000019 to 19999987
Columns: 162 entries, % Hemoglobin A1c to pH
dtypes: float64(162)
memory usage: 244.8 MB


In [27]:
df_labevents.head(50)

label,% Hemoglobin A1c,Absolute Basophil Count,Absolute CD3 Count,Absolute CD4 Count,Absolute CD8 Count,Absolute Eosinophil Count,Absolute Lymphocyte Count,Absolute Monocyte Count,Absolute Neutrophil Count,Acetaminophen,Alanine Aminotransferase (ALT),Albumin,"Albumin/Creatinine, Urine",Alkaline Phosphatase,Alpha-Fetoprotein,Amylase,Anion Gap,Anisocytosis,Anti-Thyroglobulin Antibodies,Asparate Aminotransferase (AST),Atypical Lymphocytes,Bacteria,Bands,Basophilic Stippling,Basophils,Bicarbonate,"Bilirubin, Direct","Bilirubin, Total",Blasts,C-Reactive Protein,"Calcium, Total",Calculated Free Testosterone,Calculated TBG,Carbamazepine,Carcinoembyronic Antigen (CEA),Chloride,"Cholesterol, HDL","Cholesterol, LDL, Calculated","Cholesterol, Total",Cortisol,Creatine Kinase (CK),"Creatine Kinase, MB Isoenzyme",Creatinine,Cyclosporin,DHEA-Sulfate,Digoxin,Eosinophils,Ethanol,Factor II,Factor IX,Factor V,Factor VII,Factor VIII,Factor X,Factor XI,Factor XII,Ferritin,Follicle Stimulating Hormone,Gamma Glutamyltransferase,Gentamicin,Globulin,Glucose,Granular Casts,Haptoglobin,Hematocrit,Hemoglobin,Hemoglobin A2,Hemoglobin C,Hemoglobin F,Hemogloblin S,Hepatitis B Virus Core Antibody,Homocysteine,Howell-Jolly Bodies,Human Chorionic Gonadotropin,Hyaline Casts,Hypersegmented Neutrophils,Hypochromia,INR(PT),Immunoglobulin A,Immunoglobulin G,Immunoglobulin M,Iron,Ketone,Lactate Dehydrogenase (LD),Leukocyte Alkaline Phosphatase,Lithium,Luteinizing Hormone,Lymphocytes,MCH,MCHC,MCV,Macrocytes,Magnesium,Microcytes,Monocytes,Neutrophils,Nucleated Red Cells,Other Cells,Ovalocytes,PT,PTT,Pappenheimer Bodies,Parathyroid Hormone,Phenobarbital,Phenytoin,"Phenytoin, Free",Phosphate,Platelet Count,Platelet Smear,Poikilocytosis,Polychromasia,Potassium,Procainamide,Prolactin,Promyelocytes,Prostate Specific Antigen,Protein,"Protein, Total",Protein/Creatinine Ratio,RBC,RBC Casts,RDW,Red Blood Cells,Rheumatoid Factor,Salicylate,Schistocytes,Sedimentation Rate,Serum Viscosity,Sex Hormone Binding Globulin,Sickle Cells,Sodium,Specific Gravity,Spherocytes,Target Cells,Teardrop Cells,Testosterone,"Testosterone, Free",Theophylline,Thyroglobulin,Thyroid Stimulating Hormone,Thyroxine (T4),"Thyroxine (T4), Free",Tobramycin,Transferrin,Triglycerides,Triiodothyronine (T3),Troponin T,Uptake Ratio,Urea Nitrogen,Uric Acid,"Urine Casts, Other",Urobilinogen,Valproic Acid,Vancomycin,Vitamin B12,Von Willebrand Factor Activity,Von Willebrand Factor Antigen,WBC,WBC Casts,Waxy Casts,White Blood Cells,pH
subject_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.600000,18.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.000000,34.800000,NaN,NaN,NaN,NaN,NaN,NaN,66.000000,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.300000,NaN
10000032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.833333,2.700000,NaN,111.0000,NaN,NaN,6.00,NaN,NaN,168.000000,NaN,NaN,NaN,NaN,NaN,21.000000,NaN,2.200000,NaN,NaN,7.800000,NaN,NaN,NaN,NaN,93.400000,NaN,NaN,NaN,NaN,NaN,NaN,0.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.250000,NaN,NaN,34.100000,11.566667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.933333,NaN,101.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.966667,37.600000,NaN,NaN,NaN,NaN,NaN,2.333333,106.166667,NaN,NaN,NaN,5.716667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.880000,3.430000,NaN,N

In [28]:
df_labevents.reset_index(level=0, inplace=True)

In [29]:
df_labevents.isnull().sum()

label
subject_id                              0
% Hemoglobin A1c                   183735
Absolute Basophil Count            185041
Absolute CD3 Count                 196100
Absolute CD4 Count                 195907
Absolute CD8 Count                 195943
Absolute Eosinophil Count          174316
Absolute Lymphocyte Count          171062
Absolute Monocyte Count            174204
Absolute Neutrophil Count          165933
Acetaminophen                      196199
Alanine Aminotransferase (ALT)     162474
Albumin                            158524
Albumin/Creatinine, Urine          195651
Alkaline Phosphatase               164934
Alpha-Fetoprotein                  196380
Amylase                            192123
Anion Gap                          155529
Anisocytosis                       196829
Anti-Thyroglobulin Antibodies      196787
Asparate Aminotransferase (AST)    157720
Atypical Lymphocytes               184838
Bacteria                           196827
Bands                       

In [32]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy="mean")
# imputer.fit(df_labevents_)

# imputer.statistics_

# df_labevents_.mean().values

# X = imputer.transform(df_labevents_)

# df_labevents_ = pd.DataFrame(X, columns= df_labevents_.columns,
#                      index= df_labevents_.index)

In [33]:
df_labevents_.shape

NameError: name 'df_labevents_' is not defined

In [ ]:
los_data = pd.read_csv('df_admissions_data.csv')

In [ ]:
los_data.head(10)

In [ ]:
df_labevents_['subject_id'].value_counts()

In [ ]:
# Fix NaNs and file under 'UNKNOWN'
los_data['marital_status'] = los_data['marital_status'].fillna('Unknown')
los_data['marital_status'].value_counts(dropna=False)

In [ ]:
# Compress the number of ethnicity categories

los_data['ethnicity'].replace(['UNABLE TO OBTAIN', 'OTHER',  
                         'UNKNOWN'], value='OTHER/UNKNOWN', inplace=True)

los_data['ethnicity'].value_counts()

In [ ]:
los_data.drop(columns=[ 'hadm_id','admittime', 'admission_location','discharge_location', 'language','Deceased','deathtime'], inplace=True)


In [ ]:
# Create dummy columns for categorical variables
prefix_cols = ['ADM', 'INS', 'ETH', 'MAR']
dummy_cols = ['admission_type', 'insurance','ethnicity',  'marital_status']
los_data = pd.get_dummies(los_data, prefix=prefix_cols, columns=dummy_cols)
los_data.info()

In [ ]:
LOS_ = los_data.set_index('subject_id').to_dict()['LOS']
LOS_

In [ ]:
df_labevents_['LOS'] = df_labevents_['subject_id'].map(LOS_)

In [ ]:
df_labevents_.head(50)

In [ ]:
df_labevents_['LOS'].isnull().sum()

In [ ]:
df_labevents_ = df_labevents_.fillna(0)

In [ ]:
df_labevents_ = df_labevents_[df_labevents_['LOS'] >0]

In [ ]:
df_labevents_.head(50)

In [ ]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy="mean")
# imputer.fit(df_labevents_['LOS'])

# imputer.statistics_

# df_labevents_['LOS'].mean().values

# X = imputer.transform(df_labevents_['LOS'])

# df_labevents_ = pd.DataFrame(X, columns= df_labevents_.columns,
#                      index= df_labevents_.index)

In [ ]:
df_labevents_.head(50)

In [ ]:
df_labevents_.isnull().sum()

In [ ]:
main_data = df_labevents_

In [ ]:
main_data.info()

In [ ]:
main_data.shape

In [ ]:
# main_data.to_csv('main_data.csv', index = False)

In [ ]:
main_data_corr=main_data.corr()
f,ax=plt.subplots(figsize=(18,15))
sns.heatmap(main_data_corr, cmap='viridis')
plt.title("Correlation between features", 
          weight='bold', 
          fontsize=18)
plt.xticks(weight='bold')
plt.yticks(weight='bold')

plt.show()


In [ ]:
main_data_corr["LOS"].sort_values(ascending=False)

In [ ]:
main_data.describe()

In [ ]:
main_data = main_data.drop(columns=['subject_id'])

In [ ]:
# numlist = ['Alkaline Phosphatase ', 'RDW','Lactate Dehydrogenase (LD) ','MCV','Urea Nitrogen','Monocytes', 'Potassium', 'Sodium', 'PT', 
#           'Platelet Count','Neutrophils','Eosinophils', 'Creatinine','Hemoglobin', 'Hematocrit', 'Albumin ', 'Absolute Lymphocyte Count ', 'Absolute Neutrophil Count ',
#           'RBC','C-Reactive Protein', 'WBC ','Bilirubin, Total', 'Uric Acid','Alanine Aminotransferase (ALT)', 'Globulin ',
#           'Asparate Aminotransferase (AST)','Creatine Kinase (CK)', 'Cholesterol, Total ', 'Cholesterol, HDL','Cholesterol, LDL, Calculated',
#           'Basophils', 'LOS']
# for i in main_data.columns:
#     if i in numlist:
#         upper = main_data[i].mean() + 2*main_data[i].std()
#         lower = main_data[i].mean() - 2*main_data[i].std()
#         main_data[i] = np.where(main_data[i] > upper, upper,
#                                np.where(main_data[i] < lower, lower, main_data[i]))  

In [ ]:
main_data.describe()

In [ ]:
main_data.hist(figsize=(20,15))
plt.show()

In [ ]:
main_data = main_data[main_data['Alkaline Phosphatase'] <= main_data['Alkaline Phosphatase'].quantile(0.97)]
main_data = main_data[main_data['LOS'] <= main_data['LOS'].quantile(0.97)]
print (main_data.shape) 

In [ ]:
main_data.describe()

In [ ]:
main_data.to_csv('main_data.csv', index = False)